# CE1115 Mini Project

### Group 7
Ryu, Rayne, Jia Rui
### What We Want To Explore!!
How Does The Genre Of A Movie Affects Its Chances To Win An Award?

In [52]:
# Importing neccesary libraries 
from tmdbv3api import TMDb, Movie, Discover
from tmdbv3api import Account
from tmdbv3api import Authentication
import numpy as np
import random
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle
from collections import defaultdict

In [2]:
##TMDB details
##Account details for getting movie reccomendations
##Run this block only once during a session
tmdb = TMDb()
tmdb.api_key = '0921b0cce35c0b2ec8b874614d1d0b47'
tmdb.language = 'en'

USERNAME = "cybercat94"
PASSWORD = "Wave1994@"
auth = Authentication(username=USERNAME, password=PASSWORD)
account = Account()
details = account.details()

# Oscar, Golden Globes, Bafta Datasets

In [3]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
golden_df = pd.read_csv('data/golden_globe_awards.csv')
bafta_df = pd.read_csv('data/bafta_films.csv')

## Cleaning the datasets

In [4]:
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10390,2019,2020,92,WRITING (Original Screenplay),"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True
10391,2019,2020,92,JEAN HERSHOLT HUMANITARIAN AWARD,Geena Davis,NaN,True
10392,2019,2020,92,HONORARY AWARD,David Lynch,NaN,True
10393,2019,2020,92,HONORARY AWARD,Wes Studi,NaN,True


In [5]:
golden_df

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True
...,...,...,...,...,...,...,...
7986,2019,2020,77,Best Performance by an Actor in a Supporting R...,Kieran Culkin,Succession,False
7987,2019,2020,77,Best Performance by an Actor in a Supporting R...,Andrew Scott,Fleabag,False
7988,2019,2020,77,Best Performance by an Actor in a Supporting R...,Henry Winkler,Barry,False
7989,2019,2020,77,Cecil B. deMille Award,Tom Hanks,NaN,False


In [6]:
bafta_df

,year,category,nominee,workers,winner
0,1949,Film | British Film in 1949,The Fallen Idol,NaN,True
1,1949,Film | British Film in 1949,Hamlet,NaN,False
2,1949,Film | British Film in 1949,Oliver Twist,NaN,False
3,1949,Film | British Film in 1949,Once A Jolly Swagman,NaN,False
4,1949,Film | British Film in 1949,The Red Shoes,NaN,False
...,...,...,...,...,...
4171,2020,Film | Supporting Actress in 2020,Marriage Story,Laura Dern,True
4172,2020,Film | Supporting Actress in 2020,Jojo Rabbit,Scarlett Johansson,False
4173,2020,Film | Supporting Actress in 2020,Little Women,Florence Pugh,False
4174,2020,Film | Supporting Actress in 2020,Bombshell,Margot Robbie,False


In [7]:
oscar_df= oscar_df.rename({'category':'oscar_category', 'name':'nominee', 'winner':'oscar_win'}, axis=1)
oscar_df.drop(['year_ceremony', 'ceremony'], axis=1, inplace=True)
oscar_df = oscar_df.query('year_film >=1990')

In [8]:
oscar_df['oscar_category'].unique()

array(['ACTOR IN A LEADING ROLE', 'ACTOR IN A SUPPORTING ROLE',
       'ACTRESS IN A LEADING ROLE', 'ACTRESS IN A SUPPORTING ROLE',
       'ART DIRECTION', 'CINEMATOGRAPHY', 'COSTUME DESIGN', 'DIRECTING',
       'DOCUMENTARY (Feature)', 'DOCUMENTARY (Short Subject)',
       'FILM EDITING', 'FOREIGN LANGUAGE FILM', 'MAKEUP',
       'MUSIC (Original Score)', 'MUSIC (Original Song)', 'BEST PICTURE',
       'SHORT FILM (Animated)', 'SHORT FILM (Live Action)', 'SOUND',
       'SOUND EFFECTS EDITING',
       'WRITING (Screenplay Based on Material from Another Medium)',
       'WRITING (Screenplay Written Directly for the Screen)',
       'SPECIAL ACHIEVEMENT AWARD (Visual Effects)', 'HONORARY AWARD',
       'IRVING G. THALBERG MEMORIAL AWARD', 'VISUAL EFFECTS',
       'WRITING (Screenplay Based on Material Previously Produced or Published)',
       'JEAN HERSHOLT HUMANITARIAN AWARD',
       'MUSIC (Original Dramatic Score)',
       'MUSIC (Original Musical or Comedy Score)',
       'SPECIAL 

We want to look at a smaller pool of awards.
<br> We want to have:
<br>'ACTOR IN A LEADING ROLE',
<br>'ACTOR IN A SUPPORTING ROLE',
<br>'ACTRESS IN A LEADING ROLE', 
<br>'ACTRESS IN A SUPPORTING ROLE'
<br>'DIRECTING',
<br>'CINEMATOGRAPHY',
<br>'FILM EDITING'
<br>'MUSIC (Original Score)'
<br>'BEST PICTURE',
<br>'WRITING (Screenplay Based on Material from Another Medium)',(Adapted Screenplay)
<br>'WRITING (Screenplay Written Directly for the Screen)' (Oringal Screenplay)

In [9]:
oscar_df = oscar_df.replace(to_replace={'oscar_category' : {'ACTOR IN A LEADING ROLE': 'Best Actor',
                                                'ACTOR IN A SUPPORTING ROLE' : 'Best Supporting Actor',
                                                'ACTRESS IN A LEADING ROLE': 'Best Actress',
                                                'ACTRESS IN A SUPPORTING ROLE' : 'Best Supporting Actress',
                                                'DIRECTING' : 'Best Director',
                                                'BEST PICTURE' : 'Best Pictre',
                                                'CINEMATOGRAPHY' : 'Best Cinematography',
                                                'FILM EDITING' : 'Best Film Editing',
                                                'MUSIC (Original Score)' : 'Best Original Score',
                                                'WRITING (Adapted Screenplay)' : 'Best Adapted Screenplay',
                                                'WRITING (Original Screenplay)' : 'Best Original Screenplay',},}, value=None)

oscar_df = oscar_df.query('oscar_category in ["Best Actor", "Best Supporting Actor", "Best Actress", "Best Supporting Actress", "Best Director", "Best Pictre", "Best Cinematography", "Best Film Editing", "Best Original Score","Best Adapted Screenplay","Best Original Screenplay"]')

In [77]:
oscar_df.reset_index()

,index,year_film,oscar_category,nominee,film,oscar_win
0,6889,1990,Best Actor,Kevin Costner,Dances With Wolves,False
1,6890,1990,Best Actor,Robert De Niro,Awakenings,False
2,6891,1990,Best Actor,Gerard Depardieu,Cyrano de Bergerac,False
3,6892,1990,Best Actor,Richard Harris,The Field,False
4,6893,1990,Best Actor,Jeremy Irons,Reversal of Fortune,True
...,...,...,...,...,...,...
1548,10386,2019,Best Original Screenplay,Written by Rian Johnson,Knives Out,False
1549,10387,2019,Best Original Screenplay,Written by Noah Baumbach,Marriage Story,False
1550,10388,2019,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,1917,False
1551,10389,2019,Best Original Screenplay,Written by Quentin Tarantino,Once upon a Time...in Hollywood,False


In [78]:
film = oscar_df['film']

In [87]:
discover = Discover()
movie_dict = {"id":[],"film":[]};
for i in range(1,500):
    movie = discover.discover_movies({
        'primary_release_date.gte': '1990-01-01',
        'page': i
    })
    for m in movie:
        movie_dict["id"].append(m.id)
        movie_dict["film"].append(m.original_title)
print(movie_dict)

{'id': [527774, 793723, 791373, 587807, 458576, 587996, 464052, 581389, 484718, 775996, 544401, 649087, 651571, 602269, 399566, 590706, 522444, 755812, 613504, 630586, 495764, 664767, 607383, 508442, 604822, 529203, 560144, 638597, 595813, 596247, 581387, 141052, 539885, 786300, 553604, 682254, 577922, 520946, 38700, 475557, 283566, 785534, 599281, 621876, 644092, 531499, 737568, 373571, 338762, 592350, 615677, 400160, 545609, 631939, 384018, 666750, 675490, 621013, 299536, 582306, 20982, 531219, 512200, 765169, 299534, 461130, 295830, 337401, 581734, 714277, 429617, 732450, 749618, 581392, 723072, 632632, 741228, 601666, 583689, 443791, 443791, 778730, 335983, 662334, 509967, 594718, 618353, 725649, 290859, 741074, 379291, 743439, 700995, 299537, 614409, 635302, 489245, 506281, 439213, 340382, 659067, 687149, 581032, 458156, 209112, 317442, 611914, 181812, 420817, 532067, 587496, 347201, 287947, 565743, 575088, 675327, 581600, 669444, 466622, 505225, 595149, 743904, 578189, 509635, 44

In [88]:
df = pd.DataFrame(movie_dict)
df

,id,film
0,527774,Raya and the Last Dragon
1,793723,Sentinelle
2,791373,Zack Snyder's Justice League
3,587807,Tom & Jerry
4,458576,Monster Hunter
...,...,...
9975,413765,David Lynch: The Art Life
9976,402025,Heartland
9977,419590,The Love Machine
9978,33563,Bickford Shmeckler's Cool Ideas


In [89]:
data_raw = pd.read_csv('FullDB_2021-03-15.csv')
data_raw

,Unnamed: 0,adult,id,original_title,popularity,video
0,0,False,3924,Blondie,3.837,False
1,1,False,6124,Der Mann ohne Namen,0.600,False
2,2,False,8773,L'amour à vingt ans,2.932,False
3,3,False,25449,New World Disorder 9: Never Enough,2.981,False
4,4,False,31975,Sesame Street: Elmo Loves You!,1.965,True
...,...,...,...,...,...,...
589025,589025,False,806091,The Making of 'The Pagemaster',0.000,False
589026,589026,False,806092,Kev Strikes Back,0.000,False
589027,589027,False,806093,Los Bandoleros,0.000,False
589028,589028,False,806095,Batman v Superman: Ultimate Edition - Remaster...,0.000,False


In [90]:
moivedf = pd.merge(oscar_df, df)
moivedf

,year_film,oscar_category,nominee,film,oscar_win,id
0,1990,Best Supporting Actor,Al Pacino,Dick Tracy,False,8592
1,1990,Best Cinematography,Vittorio Storaro,Dick Tracy,False,8592
2,1990,Best Actress,Kathy Bates,Misery,True,1700
3,1990,Best Actress,Julia Roberts,Pretty Woman,False,114
4,1990,Best Supporting Actress,Whoopi Goldberg,Ghost,True,251
...,...,...,...,...,...,...
1336,2019,Best Original Screenplay,"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True,291545
1337,2019,Best Film Editing,Michael McCusker and Andrew Buckland,Ford v Ferrari,True,359724
1338,2019,Best Pictre,"Peter Chernin, Jenno Topping and James Mangold...",Ford v Ferrari,False,359724
1339,2019,Best Original Score,John Williams,Star Wars: The Rise of Skywalker,False,181812
